In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [2]:
fondof = Image.open('morgana.jpg')
imgOri = Image.open("morgana.jpg")
#fondo = Image.open("fondof2.jpg")

In [3]:
#detectar objetos de un fondo dado
arrM = np.array(imgOri)
newarr= np.copy(arrM)
promfondo =np.array(fondof)
for k in range(fondof.size[0]-1):
   for c in range(fondof.size[1]-1):
     newarr[c][k][0] = abs ( arrM[c][k][0] - promfondo[c][k][0])
     newarr[c][k][1] = abs ( arrM[c][k][1] - promfondo[c][k][1])
     newarr[c][k][2] = abs ( arrM[c][k][2] - promfondo[c][k][2])
     if (newarr[c][k][0]>=255):
       newarr[c][k][0] =255
     elif(newarr[c][k][1]>=255):
      newarr[c][k][1] =255
     elif(newarr[c][k][2]>=255):
      newarr[c][k][2] =255
     if (newarr[c][k][0]<=0):
      newarr[c][k][0] = 0
     elif(newarr[c][k][1])<=0:
      newarr[c][k][1] = 0
     elif(newarr[c][k][2])<=0:
      newarr[c][k][2] = 0
    
res = Image.fromarray(newarr)
res.show()

In [4]:
img = res.convert('L')
pixels = img.getdata()
array_px = []
for pixel in pixels:
    if pixel > 128:
        array_px.append(255)
    else:
        array_px.append(0)
img_umbral = Image.new('L', img.size)
img_umbral.putdata(array_px)
img_umbral.show()
img_umbral.save("binario.jpg")


In [5]:

binario =Image.open('binario.jpg')
inicio =np.array(binario)
kernel = np.ones((3,3),np.uint8)


def dilatacion(A, B):
    m, n = A.shape
    p, q = B.shape
    C = np.zeros((m, n))

    for i in range(m):
        for j in range(n):
            for k in range(p):
                for l in range(q):
                    if B[k, l] == 1 and i+k-p//2 >= 0 and i+k-p//2 < m and j+l-q//2 >= 0 and j+l-q//2 < n:
                        C[i, j] = max(C[i, j], A[i+k-p//2, j+l-q//2])
                        

    return C

ejemplo = dilatacion(inicio,kernel)
Image.fromarray(np.uint8(ejemplo)).save('kernel_5_1.jpg')
#res = Image.fromarray(ejemplo)
#res.save('ejemplo123.jpg')
#res.show()


In [6]:
import numpy as np
from PIL import Image

def erosion(imagen, kernel):
    altura, ancho = imagen.shape
    m, n = kernel.shape
    resultado = np.zeros((altura, ancho))
    
    for i in range(m//2, altura-m//2):
        for j in range(n//2, ancho-n//2):
            # Realizar la operación de erosión
            if (imagen[i-m//2:i+m//2+1, j-n//2:j+n//2+1] * kernel).min() >= 1:
                resultado[i, j] = 1
    
    return resultado

# Ejemplo de uso
imagen = np.array(Image.open('resultado_123.jpg'))
kernel = np.ones((3,3), np.uint8)
resultado = erosion(imagen, kernel)
#res = Image.fromarray(resultado)
#res.show()
#Es el primer intento de cerrado
Image.fromarray(np.uint8(resultado*255)).save('kernel_5_2.jpg')



FileNotFoundError: [Errno 2] No such file or directory: 'resultado_123.jpg'

In [ ]:
def cerrado(A, B):
    C = dilatacion(A, B)
    D = erosion(C, B)
    return D
    
resultados = cerrado(inicio,kernel)
Image.fromarray(np.uint8(resultados*255)).save('cerrado_5_1.jpg')


In [ ]:
def abierto(A, B):
    C = erosion(A, B)
    D = dilatacion(C, B)
    return D
    
resultados = abierto(inicio,kernel)
Image.fromarray(np.uint8(resultados*255)).save('abierto_5_1.jpg')

In [ ]:
kernel = np.ones((3,3),np.uint8)
cerr = cerrado(inicio,kernel)
cerr2= abierto(cerr,kernel)
cerr3= cerrado(cerr2,kernel)
Image.fromarray(np.uint8(cerr2*255)).save('cerrabiercerr.jpg')

In [ ]:
def clear (A,B):
    C = cerrado(A, B)
    D = cerrado(C, B)
    E = abierto(D, B)
    F = erosion(E, B)
    G =erosion(F ,B)
    return G

primera = clear(inicio,kernel)
Image.fromarray(np.uint8(resultados*255)).save('clear_5_1.jpg')

In [ ]:
imgOri = Image.open("goma.jpg")
notc = Image.open("abierto_5_1.jpg")

imgris = imgOri.convert("L")
and1 = Image.new("RGB", imgOri.size)
for x in range(imgOri.width):
    for y in range(imgOri.height):
        px_cnot = notc.getpixel((x, y))
        if px_cnot == 255:
            continue
        elif px_cnot == 0:
            px_gris = imgris.getpixel((x, y))
            px_color = imgOri.getpixel((x, y))
            nuevo_pixel = (px_gris, px_color[1], px_color[2])
            and1.putpixel((x, y), nuevo_pixel)
       
and1.show()

In [ ]:
fondo = Image.open("fondoff.jpg")
img_umbral =Image.open('binario.jpg')

fondo_gr = fondo.convert("L")
and2 = Image.new("RGB", fondo.size)
for x in range(fondo.width):
    for y in range(fondo.height):
        pixel_bn = img_umbral.getpixel((x, y))
        if pixel_bn == 0:
            continue
        elif pixel_bn == 255:
            px_gris = fondo_gr.getpixel((x, y))
            px_color = fondo.getpixel((x, y))
            cambio_fondo = (px_gris, px_color[1], px_color[2])
            and2.putpixel((x, y), cambio_fondo)
and2.show()

In [ ]:
resultado = Image.new("RGB", and1.size)
for x in range(and1.width):
    for y in range(and1.height):
        px_and1 = and1.getpixel((x, y))
        px_and2 = and2.getpixel((x, y))
        if px_and1 == (0, 0, 0):
            resultado.putpixel((x, y), px_and2)
        else:
            resultado.putpixel((x, y), px_and1)
resultado.show()

In [ ]:
def erosion2(imagen, kernel):
    altura, ancho = imagen.shape
    m, n = kernel.shape
    resultado = np.zeros((altura, ancho))

    for i in range(m//2, altura-m//2):
        for j in range(n//2, ancho-n//2):
            # Realizar la operación de erosión
            if (imagen[i-m//2:i+m//2+1, j-n//2:j+n//2+1] or kernel).all() == 1:
                resultado[i, j] = 1

    return resultado
ejemplo2 = erosion2(inicio,kernel)
rest = Image.fromarray(ejemplo2)
rest.show()

In [ ]:
import cv2

# Carga la imagen en escala de grises
imagen = cv2.imread('morgana.jpg', cv2.IMREAD_GRAYSCALE)

# Aplica un filtro Gaussiano para reducir el ruido
imagen_filtrada = cv2.GaussianBlur(imagen, (5, 5), 0)

# Aplica el detector de bordes de Canny
bordes = cv2.Canny(imagen_filtrada, 100, 200)

# Muestra los bordes en una ventana
cv2.imshow('Bordes', bordes)

# Espera a que se presione una tecla
cv2.waitKey(0)

# Cierra todas las ventanas
cv2.destroyAllWindows()


In [ ]:
import cv2

# Carga la imagen en escala de grises
imagen = cv2.imread('morgana.jpg', cv2.IMREAD_GRAYSCALE)

# Aplica un filtro Gaussiano para reducir el ruido
imagen_filtrada = cv2.GaussianBlur(imagen, (5, 5), 0)

# Aplica el umbral adaptativo de Otsu
_, umbralizada = cv2.threshold(imagen_filtrada, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Encuentra los contornos en la imagen umbralizada
contornos, jerarquia = cv2.findContours(umbralizada, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Dibuja los contornos en una copia de la imagen original
imagen_con_contornos = cv2.drawContours(imagen.copy(), contornos, -1, (0, 0, 255), 2)

# Guarda la imagen con los contornos en un archivo
cv2.imwrite('siluetas.jpg', imagen_con_contornos)

# Muestra la imagen con los contornos en una ventana
cv2.imshow('Siluetas', imagen_con_contornos)

# Espera a que se presione una tecla
cv2.waitKey(0)

# Cierra todas las ventanas
cv2.destroyAllWindows()


In [ ]:
#alterar el tamaño de la imagen
from PIL import Image

# Carga la imagen
imagen = Image.open('20230316_134638.jpg')

# Redimensiona la imagen
nuevo_ancho = 200
proporcion = nuevo_ancho / imagen.width
nuevo_alto = int(imagen.height * proporcion)
imagen_redimensionada = imagen.resize((nuevo_ancho, nuevo_alto))

# Guarda la imagen redimensionada
imagen_redimensionada.save('fondoP.jpg')
